# Exercise - DTs for regression

1. Use the `./data/HousingData.csv` data (remember to split your data into a train and test data). Using your training and validation data, optimize the parameters of your DT. How well does your optimized model perform on the test data?
1. (Optional/bonus): Try to perform standardization to your data. Does it improve your model? Further, try to select only the 5 most important features. Does it improve the performance of your model?

**See slides for more details!**

In [2]:
import pandas as pd
import numpy as np
# from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn import tree

data = './data/HousingData.csv'
raw_df = pd.read_csv(data).dropna()

print(raw_df.head())

# Create a copy of the DataFrame with column names
df_copy = raw_df.copy()

# Separate the target variable (y) and features (X)
y = df_copy['MEDV']  # Replace 'TargetColumn' with your actual target column name

# MEDV - median value of owner occupied
X = df_copy.drop(columns=['MEDV'])  # Remove the target column

# Use `train_test_split` to split your data into a train and a test set.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use `train_test_split` to split your train data into a train and a validation  set.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print("\nThe shape of train, validation and test sets are:")
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD  TAX  PTRATIO  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900    1  296     15.3   
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671    2  242     17.8   
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671    2  242     17.8   
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622    3  222     18.7   
5  0.02985   0.0   2.18   0.0  0.458  6.430  58.7  6.0622    3  222     18.7   

        B  LSTAT  MEDV  
0  396.90   4.98  24.0  
1  396.90   9.14  21.6  
2  392.83   4.03  34.7  
3  394.63   2.94  33.4  
5  394.12   5.21  28.7  

The shape of train, validation and test sets are:
(252, 13) (63, 13) (79, 13) (252,) (63,) (79,)


# Exercise 1

Use the `./data/HousingData.csv` data (remember to split your data into a train and test data). Using your training and validation data, optimize the parameters of your DT. How well does your optimized model perform on the test data? Is it better than your optimized SVM for the same data (the third exercise from last week)?

In [3]:
from sklearn import tree
min_samples_split_list = [2, 4, 6, 12] # input values seperated by ",".
min_samples_leaf_list = [5, 10, 15] # input values seperated by ",".
max_features_list = [2, 5, 10] # input values seperated by ",".

results = []

for min_samples_split in min_samples_split_list:
    for min_samples_leaf in min_samples_leaf_list:
        for max_features in max_features_list:
            dt_current = tree.DecisionTreeRegressor(min_samples_split=min_samples_split,
                                                    min_samples_leaf=min_samples_leaf,
                                                    max_features=max_features)
                                                    
            dt_current.fit(X_train, y_train)
            y_val_hat = dt_current.predict(X_val)
            mse = mean_squared_error(y_val, y_val_hat)

            results.append([mse, min_samples_split, min_samples_leaf, max_features])

results = pd.DataFrame(results)
results.columns = ['MSE', 'min_samples_split', 'min_samples_leaf', 'max_features']
print(results)

          MSE  min_samples_split  min_samples_leaf  max_features
0   24.098091                  2                 5             2
1   22.470092                  2                 5             5
2   23.613533                  2                 5            10
3   27.777024                  2                10             2
4   26.611791                  2                10             5
5   19.295205                  2                10            10
6   41.788484                  2                15             2
7   26.515413                  2                15             5
8   21.356843                  2                15            10
9   37.637093                  4                 5             2
10  30.142498                  4                 5             5
11  20.315579                  4                 5            10
12  27.244707                  4                10             2
13  31.695888                  4                10             5
14  21.521898            

In [6]:
# Extract best parameters.
results[results['MSE'] == results['MSE'].max()]

print(results[results['MSE'] == results['MSE'].min()])
max_features_optimal = results.loc[results['MSE'].idxmin()]['max_features'].astype(int)
min_samples_split_optimal = results.loc[results['MSE'].idxmin()]['min_samples_split'].astype(int)
min_samples_leaf_optimal = results.loc[results['MSE'].idxmin()]['min_samples_leaf'].astype(int)

          MSE  min_samples_split  min_samples_leaf  max_features
20  13.662864                  6                 5            10


In [9]:
# Initialize your final model
dt_optimized = tree.DecisionTreeRegressor(
        max_features=max_features_optimal,
        min_samples_split=min_samples_split_optimal,
        min_samples_leaf=min_samples_leaf_optimal,
    )

# Use both training and validation data to fit it (np.concatenate "stacks" the array like rbind in R)
dt_optimized.fit(np.concatenate([X_train, X_val]), np.concatenate([y_train, y_val]))

# Predict on test data
y_test_hat_optimized = dt_optimized.predict(X_test)

# Obtain and check mse on test data
mse_optimized = mean_squared_error(y_test, y_test_hat_optimized)
print(f'Optimized DT achieved MSE = {round(mse_optimized, 2)}. - lower values indicating better performance.')



Optimized DT achieved MSE = 38.76. - lower values indicating better performance.


C:\Users\khavu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(


# Exercise 2

(Optional/bonus): Try to perform standardization to your data. Does it improve your model? Further, try to select only the 5 most important features. Does it improve the performance of your model?

In [12]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler on the training data and transform both train and test data
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(X_train_scaled.shape, X_val_scaled.shape, X_test_scaled.shape, y_train.shape, y_val.shape, y_test.shape)

(252, 13) (63, 13) (79, 13) (252,) (63,) (79,)


In [13]:
# COPY FRA SOLUTION
results_scaled = []
for min_samples_split in min_samples_split_list:
    for min_samples_leaf in min_samples_leaf_list:
        for max_features in max_features_list:
            dt_current = tree.DecisionTreeRegressor(min_samples_split=min_samples_split,
                                                    min_samples_leaf=min_samples_leaf,
                                                    max_features=max_features)
                                                    
            dt_current.fit(X_train_scaled, y_train)
            y_val_hat = dt_current.predict(X_val_scaled)
            mse = mean_squared_error(y_val, y_val_hat)
            results_scaled.append([mse, min_samples_split, min_samples_leaf, max_features])

results_scaled = pd.DataFrame(results_scaled)
results_scaled.columns = ['MSE', 'min_samples_split', 'min_samples_leaf', 'max_features']
print(results_scaled)

          MSE  min_samples_split  min_samples_leaf  max_features
0   27.167080                  2                 5             2
1   12.427780                  2                 5             5
2   21.916429                  2                 5            10
3   28.801231                  2                10             2
4   20.296075                  2                10             5
5   11.944843                  2                10            10
6   21.303572                  2                15             2
7   32.932800                  2                15             5
8   24.305137                  2                15            10
9   25.537715                  4                 5             2
10  22.110094                  4                 5             5
11  12.030186                  4                 5            10
12  18.011641                  4                10             2
13  15.676869                  4                10             5
14  18.541349            

In [14]:
# Extract best parameters.
print(results_scaled[results_scaled['MSE'] == results_scaled['MSE'].min()])
max_features_optimal = results_scaled.loc[results_scaled['MSE'].idxmin()]['max_features'].astype(int)
min_samples_split_optimal = results_scaled.loc[results_scaled['MSE'].idxmin()]['min_samples_split'].astype(int)
min_samples_leaf_optimal = results_scaled.loc[results_scaled['MSE'].idxmin()]['min_samples_leaf'].astype(int)

         MSE  min_samples_split  min_samples_leaf  max_features
5  11.944843                  2                10            10


In [16]:
# from sklearn.tree import DecisionTreeRegressor
# Initialize your final model
final_dt =  tree.DecisionTreeRegressor(
                max_features=max_features_optimal,
                min_samples_split=min_samples_split_optimal,
                min_samples_leaf=min_samples_leaf_optimal,
                )   

# Use both training and validation data to fit it (np.concatenate "stacks" the array like rbind in R)
final_dt.fit(np.concatenate([X_train, X_val]), np.concatenate([y_train, y_val]))

# Predict on test data
y_test_hat_optimized = final_dt.predict(X_test)

# Obtain and check mse on test data
rf_optimized = mean_squared_error(y_test, y_test_hat_optimized)
print(f'Optimized RF achieved MSE = {round(rf_optimized, 1)}. - lower values indicating better performance.')


Optimized RF achieved MSE = 32.9. - lower values indicating better performance.


C:\Users\khavu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
